# TaskA RDD based approach

In [1]:
serversPath = "/user/s292129/data/exam_ex5_data/Servers.txt"
temperaturesAnomaliesPath = "/user/s292129/data/exam_ex5_data/Servers_TemperatureAnomalies.txt"
outputPath1 = "exam_ex5_out1/"
outputPath2 = "exam_ex5_out2/"

In [2]:
anomInputRDD = sc.textFile(temperaturesAnomaliesPath).cache()
# schema: sID, TimeStamp (date_time), AnomTempValue

In [3]:
filteredAnomRDD = anomInputRDD.filter(lambda line: int(line.split(',')[1].split('_')[0].split("/")[0])\
                    in range(2010, 2019) and float(line.split(',')[2]) > 100)

In [4]:
def returnPair(line):
    fields = line.split(",")
    sID = fields[0]
    year = fields[1].split("_")[0].split("/")[0]
    return ((sID, year), 1)
    
sIdYear_oneRDD = filteredAnomRDD.map(returnPair)

In [5]:
sIdYear_numAnomaliesRDD = sIdYear_oneRDD.reduceByKey(lambda v1, v2: v1+v2)

In [6]:
filteredPerYearRDD = sIdYear_numAnomaliesRDD.filter(lambda pair: pair[1] > 2)

In [7]:
filteredPerYearRDD.collect()

[(('SID13', '2018'), 3), (('SID13', '2017'), 3)]

In [9]:
filteredPerYearRDD.map(lambda pair: pair[0][0]).distinct().collect()

['SID13']

# Task B - ERRATO

In [13]:
filteredAnom2_RDD = anomInputRDD.filter(lambda line: int(line.split(',')[1].split('_')[0].split("/")[0])\
                    in range(2010, 2019))

In [14]:
serversRDD = sc.textFile(serversPath)
def returnPairs(line):
    fields = line.split(",")
    sID = fields[0]
    dcID = fields[2]
    return (sID, dcID)

sID_dcID_RDD = serversRDD.map(returnPairs)

In [16]:
def returnPair2(line):
    fields = line.split(",")
    sID = fields[0]
    return (sID, 1)

sID_oneRDD = filteredAnom2_RDD.map(returnPair2)

In [18]:
sID_numAnomaliesRDD = sID_oneRDD.reduceByKey(lambda v1, v2: v1+v2)

In [22]:
filteredPerAnomRDD = sID_numAnomaliesRDD.filter(lambda pair: pair[1] <= 3)

In [24]:
jointRDD = sID_dcID_RDD.join(filteredPerAnomRDD)
# schema: sID, dcID, numAnom


In [26]:
jointRDD = jointRDD.map(lambda pair: (pair[1][0], pair[0]))

dcID_RDD = sID_dcID_RDD.map(lambda pair: (pair[1], pair[0]))


In [ ]:
# we can cogroup on dcID and compare the size of the two lists
cogroupedRDD = jointRDD.cogroup(dcID_RDD)

In [ ]:
def finalFilter(pair):
    l1 = list(pair[1][0])
    l2 = list(pair[1][1])
    if(len(l1) == len(l2)):
        return True
    
cogroupedRDD.filter(finalFilter).keys().saveAsTextFile(outputPath2)